In [2]:
from datasets import load_from_disk, load_metric, Dataset
from transformers import RobertaTokenizer, DataCollatorWithPadding, RobertaForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import pyarrow as pa
import pandas as pd

In [3]:
# load
datadir = '/work3/s174498/sst2_dataset/'
checkpoint = "/work3/s174498/finetuning-sentiment-model-all-samples-test5/checkpoint-13000"

# test data
test_dataset = load_from_disk(datadir + 'test_dataset')

# tokenizer
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)

# model
model = RobertaForSequenceClassification.from_pretrained(checkpoint)

In [4]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [5]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /work3/s174498/sst2_dataset/test_dataset/cache-ef0affb690c3e1df.arrow


In [16]:
trainer = Trainer(
    model=model,                        
    tokenizer=tokenizer
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [8]:
# Predicting with model
predictions = trainer.predict(tokenized_test)
dataset_test_pred = list(np.argmax(predictions.predictions, axis=-1))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8


In [15]:
# compute accuracy
accuracy_metric = load_metric("accuracy")
accuracy_metric.compute(predictions=dataset_test_pred, references=test_dataset['label'])

{'accuracy': 0.9890170236133993}